# CABIN Predictor Analysis
DEM- 25m from warehouse
Climate data- percipation total(PPT) and temp max(Tmax) rasters from https://climatebc.ca/
Land cover data(2000) from - https://ftp.maps.canada.ca/pub/nrcan_rncan/vector/geobase_lcc_csc/shp_en/093/lcc2000v_csc2000v_093a_shp_en.zip
Landcover classifications- https://ftp.maps.canada.ca/pub/nrcan_rncan/vector/geobase_lcc_csc//doc/GeoBase_lcc_en_Catalogue.pdf
Catchment areas created from pysheds

In [ ]:
import os 
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import geopandas as gpd
import pandas as pd 
import numpy as np
import rasterio as rio
import rasterio.mask
from shapely.geometry import shape, box


In [8]:
output_table_loc=os.path.join("..","deliverables","model_predictors3.xlsx")
catchments=os.path.join("..","work","catchments.gpkg")
catchments_lyrs=gpd.list_layers(catchments)
catchments_lyrs=catchments_lyrs['name'].tolist()
workdir=os.path.join("..","work")
precip_rasters=glob(os.path.join(workdir,"PPT*"))
temp_rasters=glob(os.path.join(workdir,"Tmax*"))
landclass_shp=os.path.join("..","source_data","landcover_2000","LCC2000-V_093A_2_0.shp")
slope_loc=os.path.join("..","source_data","slope_clipped_25m.tif")

In [ ]:
landclass=gpd.read_file(landclass_shp)
landclass=landclass.to_crs(3005)

In [ ]:
cols=["site","catchment_area_ha", "Natl-SnowIce", "Natl-Water", "Precip06_JUN", "Precip08_AUG", "Precip12_DEC", "Slope30-50%", "Temp06_JUNMax",
      "Temp08_AUGMax", "Temp12_DECMax"]
output_table = pd.DataFrame(columns=cols)

In [ ]:
for f in catchments_lyrs:
    catchment=gpd.read_file(catchments, layer=f)

    site=catchment['site'][0]
    area=catchment.area[0]/10000

    lc=gpd.clip(landclass, catchment)
    lc=lc.dissolve('COV_LABEL')
    if 'Water' in lc.index:
        water=(lc.loc['Water'].geometry.area/10000)/area*100
        print(f"water percentage in {site}: {water:.2f}%")
    else:
        water=0
        print('no water')
        
    if 'Snow/Ice' in lc.index:
        snow_ice=(lc.loc['Snow/Ice'].geometry.area/10000)/area*100
        print(f"snow/ice percentage in {site}: {snow_ice:.2f}%")
    else:
        snow_ice=0
        print('no snow/ice, changing to 0 percent')
    #Get area of slope between 30-50%
    with rio.open(slope_loc) as slope:
        catchment=catchment.to_crs(slope.crs)
        out_image, out_transform = rasterio.mask.mask(slope, catchment.geometry, crop=True)
        out_meta = slope.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        # Create mask for values between 30 and 50 (inclusive)
        slope_mask = (out_image[0] >= 30) & (out_image[0] <= 50)
        
        # Count cells that meet the criteria
        slope_cell_count = np.sum(slope_mask)
        
        # Calculate pixel area
        pixel_x_size = abs(out_transform[0])
        pixel_y_size = abs(out_transform[4])
        pixel_area_m2 = pixel_x_size * pixel_y_size
        
        # Calculate total area in hectares
        slope_area_ha = (slope_cell_count * pixel_area_m2) / 10000
        
        # Calculate percentage of catchment area
        slope_percentage = (slope_area_ha / area) * 100
        
        print(f"Area with slope 30-50%: {slope_area_ha} ha ({slope_percentage}% of catchment)")

    rast_list=precip_rasters+temp_rasters


    for rast in rast_list:
        with rio.open(rast) as src:
            catchment=catchment.to_crs(src.crs)
            bbox = catchment.bounds
            bbox_geom = box(bbox.minx.iloc[0], bbox.miny.iloc[0], bbox.maxx.iloc[0], bbox.maxy.iloc[0])
            out_image, out_transform = rasterio.mask.mask(src,[bbox_geom], crop=True)
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })
            valid_data = out_image[out_image != src.nodata]
            valid_data = valid_data[np.isfinite(valid_data)]
            mean_value=valid_data.mean()

            plt.tight_layout()
            plt.show()
            
            if 'PPT' in os.path.basename(rast):
                if '06' in os.path.basename(rast):
                    ppt06_mean = mean_value
                    print(f"Mean Precip06_JUN in {site}: {ppt06_mean:.2f} mm")
                elif '08' in os.path.basename(rast):
                    ppt08_mean = mean_value
                    print(f"Mean Precip08_AUG in {site}: {ppt08_mean:.2f} mm")
                elif '12' in os.path.basename(rast):
                    ppt12_mean = mean_value
                    print(f"Mean Precip12_DEC in {site}: {ppt12_mean:.2f} mm")
            elif 'Tmax' in os.path.basename(rast):
                if '06' in os.path.basename(rast):
                    tmax06_mean = mean_value
                    print(f"Mean Temp06_JUNMax in {site}: {tmax06_mean:.2f} °C")
                elif '08' in os.path.basename(rast):
                    tmax08_mean = mean_value
                    print(f"Mean Temp08_AUGMax in {site}: {tmax08_mean:.2f} °C")
                elif '12' in os.path.basename(rast):
                    tmax12_mean = mean_value
                    print(f"Mean Temp12_DECMax in {site}: {tmax12_mean:.2f} °C")
                    
    add_data=[site, area, snow_ice, water, ppt06_mean, ppt08_mean, ppt12_mean, slope_percentage, tmax06_mean, tmax08_mean, tmax12_mean]
    output_table.loc[len(output_table)] = add_data

In [9]:
output_table.to_excel(output_table_loc)